In [51]:
import altair as alt
import pandas as pd
import os
from utils.find_root import find_project_root

In [53]:
# Retrieve the project root dynamically and set it as working directory
project_root = find_project_root()
os.chdir(project_root)
# REPLACE WITH THE CORRECT PATH
df = pd.read_csv("outputs/xai/dashboard_input_20250531_1700_xai_enhanced.csv", parse_dates=["date"], index_col='date').reset_index()
# Define dashboard visuals directory
DASHBOARD_VISUALS_DIR = "outputs/dashboard/inference_visuals/"

# Ensure output directory exist
os.makedirs(DASHBOARD_VISUALS_DIR, exist_ok=True)
df

,date,temperature_2m,surface_pressure,precipitation,wind_speed_10m,temp_lower,temp_upper,wind_lower,wind_upper,press_lower,...,is_if_anomaly,lstm_error,is_lstm_anomaly,if_threshold,lstm_threshold,anomaly_label,shap_summary,shap_plot_path,reconstruction_error_summary,reconstruction_error_plot
0,2025-05-31 17:00:00,23.0,1011.4,0.0,24.1,10,26.25,4.1,27.1,1000.160075,...,0,0.680835,1,0.033044,0.6425,Pattern anomaly,The model's baseline anomaly score is **12.70*...,explanations/plots/row_0.png,🧭 Anomaly Type: Unusual sequence pattern (patt...,/content/drive/MyDrive/Data Science Group Proj...
1,2025-05-31 18:00:00,21.9,1011.2,0.0,24.8,10,26.25,4.1,27.1,1000.160075,...,0,0.644328,1,0.033044,0.6425,Pattern anomaly,The model's baseline anomaly score is **12.70*...,explanations/plots/row_1.png,🧭 Anomaly Type: Unusual sequence pattern (patt...,/content/drive/MyDrive/Data Science Group Proj...
2,2025-05-31 19:00:00,21.1,1010.8,0.0,25.2,10,26.25,4.1,27.1,1000.160075,...,0,0.668963,1,0.033044,0.6425,Pattern anomaly,The model's baseline anomaly score is **12.70*...,explanations/plots/row_2.png,🧭 Anomaly Type: Unusual sequence pattern (patt...,/content/drive/MyDrive/Data Science Group Proj...
3,2025-05-31 20:00:00,20.1,1010.9,0.0,15.8,10,26.25,4.1,27.1,1000.160075,...,0,0.507134,0,0.033044,0.6425,Normal,The model's baseline anomaly score is **12.70*...,explanations/plots/row_3.png,🧭 Anomaly Type: No concern\n\nNo significant a...,/content/drive/MyDrive/Data Science Group Proj...
4,2025-05-31 21:00:00,18.7,1011.7,0.0,18.4,10,26.25,4.1,27.1,1000.160075,...,0,0.508397,0,0.033044,0.6425,Normal,The model's baseline anomaly score is **12.70*...,explanations/plots/row_4.png,🧭 Anomaly Type: No concern\n\nNo significant a...,/content/drive/MyDrive/Data Science Group Proj...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,2025-06-03 12:00:00,15.3,1001.4,0.1,27.4,10,26.25,4.1,27.1,1000.160075,...,1,0.709597,1,0.033044,0.6425,Compound anomaly,The model's baseline anomaly score is **12.70*...,explanations/plots/row_67.png,🧭 Anomaly Type: Aligned signal from both model...,/content/drive/MyDrive/Data Science Group Proj...
68,2025-06-03 13:00:00,16.1,1001.0,0.1,27.0,10,26.25,4.1,27.1,1000.160075,...,1,0.757791,1,0.033044,0.6425,Compound anomaly,The model's baseline anomaly score is **12.70*...,explanations/plots/row_68.png,🧭 Anomaly Type: Aligned signal from both model...,/content/drive/MyDrive/Data Science Group Proj...
69,2025-06-03 14:00:00,16.9,1000.9,0.9,25.6,10,26.25,4.1,27.1,1000.160075,...,1,0.842720,1,0.033044,0.6425,Compound anomaly,The model's baseline anomaly score is **12.70*...,explanations/plots/row_69.png,🧭 Anomaly Type: Aligned signal from both model...,/content/drive/MyDrive/Data Science Group Proj...
70,2025-06-03 15:00:00,17.7,1000.9,0.9,23.4,10,26.25,4.1,27.1,1000.160075,...,1,0.832518,1,0.033044,0.6425,Compound anomaly,The model's baseline anomaly score is **12.70*...,explanations/plots/row_70.png,🧭 Anomaly Type: Aligned signal from both model...,/content/drive/MyDrive/Data Science Group Proj...


In [54]:
### Precipitation Chart ###

# Set flexible y-axis upper bound
y_max = max(6, df["precipitation"].max())

# Define anomaly domain (even if some are missing)
anomaly_domain = ['Point anomaly', 'Pattern anomaly', 'Compound anomaly']

# Base chart config
base = alt.Chart(df).encode(
    x=alt.X('date:T',
            title='Date & Time',
            axis=alt.Axis(format='%d %b %H:%M', labelAngle=-45, tickCount=12,grid=False))
)

# Line plot for precipitation
precip_line = base.mark_line(color='steelblue').encode(
    y=alt.Y('precipitation:Q', title='Precipitation (mm)', scale=alt.Scale(domain=[0, y_max]))
)

# Threshold rules with dummy categories
thresholds_df = pd.DataFrame({
    'y': [0.5, 2.0, 5.0],
    'label': ['Light Rain (0.5mm)', 'Moderate Rain (2mm)', 'Heavy Rain (5mm)']
})

threshold_lines = alt.Chart(thresholds_df).mark_rule(strokeDash=[4, 2]).encode(
    y='y:Q',
    color=alt.Color('label:N',
                    scale=alt.Scale(domain=thresholds_df['label'].tolist(),
                                    range=['green', 'orange', 'red']),
                    title='Rain Intensity Thresholds')
)

# Anomaly circles — using same logic as your original, but force full legend
anomalies = base.mark_circle(size=60).encode(
    y='precipitation:Q',
    color=alt.Color('anomaly_label:N',
    scale=alt.Scale(
        domain=['Point anomaly', 'Pattern anomaly', 'Compound anomaly'],
        range=['#00bfff', '#ba55d3', '#27408b']
    ),
    title='Anomaly Type'
),
    tooltip=[
        alt.Tooltip('date:T', title='Timestamp', format='%d %b %H:%M'),
        alt.Tooltip('precipitation:Q', title='Precipitation (mm)'),
        alt.Tooltip('anomaly_label:N', title='Anomaly Type')
    ]
).transform_filter(
    alt.datum.anomaly_label != 'Normal'
)

# Compose layers and retain separate legends
final_precip_chart = alt.layer(
    precip_line,
    threshold_lines,
    anomalies
).resolve_scale(
    color='independent'
).properties(
    title='72-Hour Precipitation Forecast: Anomalies and Rain Thresholds',
    width=900,
    height=400
)

# Save chart
final_precip_chart.save(f"{DASHBOARD_VISUALS_DIR}precip_timeline_plot.html")

In [55]:
### Temperature Chart ###

# Y-axis limits
y_max_temp = max(df["temperature_2m"].max(), df["temp_upper"].max()) + 1
y_min_temp = min(df["temperature_2m"].min(), df["temp_lower"].min()) - 1

# Add label for band legend
df["temp_band_label"] = "Normal Range (Q1 to Q3 + 1.5×IQR)"

# Base chart
base_temp = alt.Chart(df).encode(
    x=alt.X('date:T',
            title='Date & Time',
            axis=alt.Axis(format='%d %b %H:%M', labelAngle=-45, tickCount=12,grid=False))
)

# Temperature line
temp_line = base_temp.mark_line(color='steelblue').encode(
    y=alt.Y('temperature_2m:Q',
            title='Temperature (°C)',
            scale=alt.Scale(domain=[y_min_temp, y_max_temp]))
)

# Temperature band with legend
temp_band = base_temp.mark_area(opacity=0.4).encode(
    y='temp_lower:Q',
    y2='temp_upper:Q',
    color=alt.Color('temp_band_label:N',
                    scale=alt.Scale(domain=['Normal Range (Q1 to Q3 + 1.5×IQR)'],
                                    range=['lightgrey']),
                    legend=alt.Legend(title='Temperature Band (last 60 days)'))
)

# Anomaly dots
anomalies_temp = base_temp.mark_circle(size=60).encode(
    y='temperature_2m:Q',
    color=alt.Color('anomaly_label:N',
                    scale=alt.Scale(
                        domain=['Point anomaly', 'Pattern anomaly', 'Compound anomaly'],
                        range=['#00bfff', '#ba55d3', '#27408b']),
                    title='Anomaly Type'),
    tooltip=[
        alt.Tooltip('date:T', title='Timestamp', format='%d %b %H:%M'),
        alt.Tooltip('temperature_2m:Q', title='Temperature (°C)'),
        alt.Tooltip('anomaly_label:N', title='Anomaly Type')
    ]
).transform_filter(
    alt.datum.anomaly_label != 'Normal'
)

# Compose final temperature chart
final_temp_chart = alt.layer(
    temp_band,
    temp_line,
    anomalies_temp
).resolve_scale(
    color='independent'
).properties(
    title='72-Hour Temperature Forecast: Anomalies and Confidence Band',
    width=900,
    height=400
)

# Save
final_temp_chart.save(f"{DASHBOARD_VISUALS_DIR}temperature_timeline_plot.html")

In [56]:
### Wind Speed Chart ###

# Y-axis limits
y_max_wind = max(df["wind_speed_10m"].max(), df["wind_upper"].max()) + 1
y_min_wind = min(df["wind_speed_10m"].min(), df["wind_lower"].min()) - 1

# Add band label for legend
df["wind_band_label"] = "Normal Range (10th to Q3 + 1.5×IQR)"

# Base chart
base_wind = alt.Chart(df).encode(
    x=alt.X('date:T',
            title='Date & Time',
            axis=alt.Axis(format='%d %b %H:%M', labelAngle=-45, tickCount=12,grid=False))
)

# Wind line
wind_line = base_wind.mark_line(color='steelblue').encode(
    y=alt.Y('wind_speed_10m:Q',
            title='Wind Speed (km/h)',
            scale=alt.Scale(domain=[y_min_wind, y_max_wind]))
)

# Wind band
wind_band = base_wind.mark_area(opacity=0.4).encode(
    y='wind_lower:Q',
    y2='wind_upper:Q',
    color=alt.Color('wind_band_label:N',
                    scale=alt.Scale(domain=['Normal Range (10th to Q3 + 1.5×IQR)'],
                                    range=['lightgrey']),
                    legend=alt.Legend(title='Wind Speed Band (last 60 days)'))
)

# Anomalies (shared legend)
anomalies_wind = base_wind.mark_circle(size=60).encode(
    y='wind_speed_10m:Q',
    color=alt.Color('anomaly_label:N',
                    scale=alt.Scale(
                        domain=['Point anomaly', 'Pattern anomaly', 'Compound anomaly'],
                        range=['#00bfff', '#ba55d3', '#27408b']),
                    title='Anomaly Type'),
    tooltip=[
        alt.Tooltip('date:T', title='Timestamp', format='%d %b %H:%M'),
        alt.Tooltip('wind_speed_10m:Q', title='Wind Speed (km/h)'),
        alt.Tooltip('anomaly_label:N', title='Anomaly Type')
    ]
).transform_filter(
    alt.datum.anomaly_label != 'Normal'
)

# Compose final wind chart
final_wind_chart = alt.layer(
    wind_band,
    wind_line,
    anomalies_wind
).resolve_scale(
    color='independent'
).properties(
    title='72-Hour Wind Speed Forecast: Anomalies and Confidence Band',
    width=900,
    height=400
)

# Save
final_wind_chart.save(f"{DASHBOARD_VISUALS_DIR}wind_timeline_plot.html")

In [57]:
### Pressure Chart ###

# Add legend label for band
df["press_band_label"] = "Normal Range (±2×std)"

# Pressure band (x-axis: 12 ticks, no vertical grid; y-axis: fixed domain, grid ON)
press_band = alt.Chart(df).mark_area(opacity=0.2).encode(
    x=alt.X('date:T',
            axis=alt.Axis(format='%d %b %H:%M', title='Date & Time',labelAngle=-45, tickCount=12, grid=False)),
    y=alt.Y('press_lower:Q',
            scale=alt.Scale(domain=[980, 1050])),
    y2='press_upper:Q',
    color=alt.Color('press_band_label:N',
                    scale=alt.Scale(domain=['Normal Range (±2×std)'],
                                    range=['lightgrey']),
                    legend=alt.Legend(title='Surface Pressure Band (last 60 days)'))
)

# Pressure line
press_line = alt.Chart(df).mark_line(color='steelblue').encode(
    x=alt.X('date:T',
            axis=alt.Axis(format='%d %b %H:%M', labelAngle=-45, tickCount=12, grid=False)),
    y=alt.Y('surface_pressure:Q',
            scale=alt.Scale(domain=[980, 1050]),
            title='Surface Pressure (hPa)')
)

# Anomaly dots
anomalies_press = alt.Chart(df[df["anomaly_label"] != "Normal"]).mark_circle(size=60).encode(
    x=alt.X('date:T',
            axis=alt.Axis(format='%d %b %H:%M', labelAngle=-45, tickCount=12, grid=False)),
    y=alt.Y('surface_pressure:Q',
            scale=alt.Scale(domain=[980, 1050])),
    color=alt.Color('anomaly_label:N',
                    scale=alt.Scale(
                        domain=['Point anomaly', 'Pattern anomaly', 'Compound anomaly'],
                        range=['#00bfff', '#ba55d3', '#27408b']),
                    title='Anomaly Type'),
    tooltip=[
        alt.Tooltip('date:T', title='Timestamp', format='%d %b %H:%M'),
        alt.Tooltip('surface_pressure:Q', title='Surface Pressure (hPa)'),
        alt.Tooltip('anomaly_label:N', title='Anomaly Type')
    ]
)

# Compose final chart
final_press_chart = alt.layer(
    press_band,
    press_line,
    anomalies_press
).resolve_scale(
    y='shared',
    color='independent'
).properties(
    title='72-Hour Surface Pressure Forecast: Anomalies and Confidence Band',
    width=900,
    height=400
)

# Save chart
final_press_chart.save(f"{DASHBOARD_VISUALS_DIR}pressure_timeline_plot.html")

In [58]:
### Stacked Chart for All Variables ###

# Stack charts without caption
final_dashboard_chart = alt.vconcat(
    final_precip_chart,
    final_temp_chart,
    final_wind_chart,
    final_press_chart
).resolve_scale(
    x='shared',
    color='independent'
).configure_axis(
    grid=True
).configure_view(
    stroke=None
)

# Save to file
final_dashboard_chart.save(f"{DASHBOARD_VISUALS_DIR}combined_dashboard_timeline.html")

In [59]:
### Model Scores Plot - Expert Mode ###

# Y-axis bounds with small padding
y_min = df['if_score'].min() - 0.04
y_max = max(df['lstm_error'].max(), df['if_score'].max()) + 0.045

# Thresholds
lstm_thresh = df["lstm_threshold"].iloc[0]
if_thresh = df["if_threshold"].iloc[0]

# Red band overlays
band_df = pd.DataFrame({
    "date": [df["date"].min(), df["date"].max()],
    "lstm_threshold": [lstm_thresh] * 2,
    "lstm_top": [y_max] * 2,
    "if_threshold": [if_thresh] * 2,
    "if_bottom": [y_min] * 2,
    "zone_type": ["Threshold Breach Zone"] * 2
})

top_band = alt.Chart(band_df).mark_area(opacity=0.15).encode(
    x='date:T',
    y='lstm_threshold:Q',
    y2='lstm_top:Q',
    color=alt.Color('zone_type:N',
        scale=alt.Scale(domain=['Threshold Breach Zone'], range=['red']),
        legend=alt.Legend(title='Shaded Zone'))
)

bottom_band = alt.Chart(band_df).mark_area(opacity=0.15).encode(
    x='date:T',
    y='if_bottom:Q',
    y2='if_threshold:Q',
    color=alt.Color('zone_type:N',
        scale=alt.Scale(domain=['Threshold Breach Zone'], range=['red']),
        legend=None)
)

# Model score lines
lstm_line = alt.Chart(df).mark_line(color='#ba55d3').encode(
    x=alt.X('date:T', axis=alt.Axis(format='%d %b %H:%M', tickCount=12, labelAngle=-45, grid=False)),
    y=alt.Y('lstm_error:Q', title='Score', scale=alt.Scale(domain=[y_min, y_max]))
)

if_line = alt.Chart(df).mark_line(color='#00bfff').encode(
    x='date:T',
    y='if_score:Q'
)

# Rule lines for thresholds
lstm_thresh_line = alt.Chart(df).mark_rule(strokeDash=[4, 2], color='#ba55d3').encode(
    y='lstm_threshold:Q'
)
if_thresh_line = alt.Chart(df).mark_rule(strokeDash=[4, 2], color='#00bfff').encode(
    y='if_threshold:Q'
)

# Prepare anomaly dots with new source labels
df_lstm_anom = df[df["is_lstm_anomaly"] == True].copy()
df_lstm_anom["source"] = "LSTM Anomaly"
df_lstm_anom["y_val"] = df_lstm_anom["lstm_error"]

df_if_anom = df[df["is_if_anomaly"] == True].copy()
df_if_anom["source"] = "IF Anomaly"
df_if_anom["y_val"] = df_if_anom["if_score"]

df_dots = pd.concat([df_lstm_anom, df_if_anom])

# Plot anomaly dots with unified legend
dots_combined = alt.Chart(df_dots).mark_circle(size=60).encode(
    x='date:T',
    y='y_val:Q',
    color=alt.Color('source:N',
        scale=alt.Scale(domain=["LSTM Anomaly", "IF Anomaly"], range=['#ba55d3', '#00bfff']),
        legend=alt.Legend(title='Anomaly Type')),
    tooltip=[
        alt.Tooltip('date:T', title='Timestamp', format='%d %b %H:%M'),
        alt.Tooltip('y_val:Q', title='Score'),
        alt.Tooltip('anomaly_label:N', title='Anomaly Label')
    ]
)

# Combine all layers
final_score_chart = alt.layer(
    top_band,
    bottom_band,
    lstm_line,
    if_line,
    lstm_thresh_line,
    if_thresh_line,
    dots_combined
).resolve_scale(
    color='independent'
).properties(
    title='LSTM Error & IF Score with Threshold Zones and Anomalies',
    width=900,
    height=400
)

# Save
final_score_chart.save("outputs/dashboard/inference_visuals/lstm_if_scores_final_banded.html")update